# 3. Artificial Neural Networks 

Artificial Neural Networks (ANNs) provide a versatile way to build any machine learning model. The idea is inspired by the structure and functioning of the human brain. Neural networks have proven to be powerful for solving complex problems of classification and other tasks. In this class, we will explain the basics of neural networks, their components, training process and validation of ANNs.

There are several libraries for Python which implement ANN. The most known are [Tensorflow](https://www.tensorflow.org) and [PyTorch](https://pytorch.org) (there is also a library which takes the best of both, [Keras](https://keras.io)). For the examples in this class we will use PyTorch, which is, in these days, probably the most widely used and well documented. 

Install the library by running the following code (after that add `#` in front of this line to avoid running it again):
 

In [ ]:
! pip install torch torchinfo

As you may noticed we installed two libraries here, the PyTorch (`torch`) and a supplement which will help us to discover the structure of ANNs (`torchinfo`).

Before we continue, let's split our data to the training and test set again, as we did it in the previous class. We will later use the training set for the learning process and test set to assess how good the trained model is.

Strictly speaking in this case we need three sets:

* *training set* is used for learning
* *validation set* is used to optimize the model (find the best one during learning)
* *test set* is used to assess quality of the final model

In order to simplify examples we will use *test set* both for validation and testing, but when you  work with real cases, make sure that they are separated and you have all three sets (we will use this approach in the next class).

Like in the previous class we simply take every fifth measurement as a test set and later we will learn how to make it in a better way.

In [ ]:
# load data from CSV file as data frame
import pandas as pd
d = pd.read_csv("Iris.csv")

# generate logical values for train and test set measurements (rows of data frame)
train_ind = d["Id"] % 5 != 0
test_ind = d["Id"] % 5 == 0

# make the split
d_train = d.loc[train_ind]
d_test = d.loc[test_ind]

# show size of each set
(d_train.shape, d_test.shape)

#### What is a neural network?

A neural network is simply a set of nodes, known as *neurones*, which are connected to each other. In the simplest case, neural network consists of only one node as shown in the image below.

<img src="./illustrations/Neuron.png" style="width:500px; height:400px;"/>

Every neuron has a set of inputs (shown as $X_1$, $X_2$, $X_3$, and $X_4$ on the left part of the image) and the output (shown as $\hat{Y}$ on the right part). It does a very simle thing — takes all numbers, which it receives from the inputs, and applies a mathematical function, which computes an output value based on the inputs. 

In the simplest case it computes  a weighted sum (known as *linear combination*) of these numbers, and transmits the computed number to the output. Mathematically we can write this as follows:

$\hat{Y} = (X_{1}\times W_{1}) + (X_{2}\times W_{2}) + (X_{3}\times W_{3}) + (X_{4}\times W_{4}) + Bias$

The values $W_1$,...,$W_4$ are *weights* every input contribute to the output with.

Imagine, that the inputs are the measurements  from the first row of the Iris dataset: 

$X = [5.1,3.5,1.4,0.2]$ 

and the weights are: 

$W = [0.1, 0.2, 2.0, 5.0]$. 

Let's assume that the bias is $1.0$. Then the output value for our neuron will be:

$\hat{Y} = 5.1 \times 0.1 + 3.5 \times 0.2 + 1.4 \times 2.0 + 0.2 \times 5.0 + 1.0 = 6.01$

As simple as that.

Here is how we can implement this one neuron based ANN in Python using PyTorch library:

In [ ]:
import torch
import torch.nn as nn

class SimpleModel(nn.Module):
    """ class for one-neuron ANN model """

    def __init__(self):
        # initialize the parent (super) class for ANN model
        super(SimpleModel, self).__init__()
        # define all layers with neurons and their properties
        self.layer1 = nn.Linear(4, 1)

    def forward(self, x):
        """ takes vector with input values 'x', computes and returns the output """
        y_hat = self.layer1(x)
        return y_hat

As you can see, the code is a bit more complex comparing to what we have had before. In this code we create a *class* `SimpleModel` on top of another class `nn.Module`, which is already implemented in PyTorch. 

>You can think about class as a recipe, a blue print, a set of instructions. Imagine you want to build a house. There is a set of basic instructions which tell how to build a foundation of the house, set drainage pipes etc. Plus it has some typical instructions, e.g. how to make a brick wall. There is no need to reinvent the wheel, so when you want to make instructions for a building a full house (a typical project), with walls, roof, etc. you can take the basic set as a basis and extend it with your own part. 

Same idea here, the `nn.Model` is a basic set of instructions how to make any neural network in PyTorch, and it includes a lot of things you do not need to care of. You simply take it as a basis and extend with your own specific parts — which nodes to use, how many inputs they have, how many outputs, etc. So this new set of instructions is the class `SimpleModel`.

As you can see, we added two methods to the class. 

The first method, `__init__` is needed to initialize your model. At the beginning it contains:

```python
super(SimpleModel, self).__init__()
``` 

which tells Python to make all preparations based on the basic set of instructions (e.g. make foundation of the house).

And then you define your network — how many neurons, what kind of neurons and define their properties. In this case, we have one linear neuron with 4 inputs and 1 output — exactly what we used in the example above. 

**You can think of `__init__()` as a method that builds the house using your instructions.** Not painted, without furniture, but a whole, fully functioning house. 

The second method in this class, `forward()`, is used every time you want to apply your model for given inputs to produce the output. The method "connects" the neurons, it makes sure that the inputs go through the neurons in a correct order and returns the resulted output at the end.  

**You can think of `forward()` as a method that lets you use the house you have created.** It is always used in its current state. So if you apply method `forward()` to newly created house, it will not perform well as your house has bare walls and no furniture. But you can live there of course.

Here is an example how we can use the model:

In [ ]:
# initialize the model
model = SimpleModel()

# define values for input
X = torch.tensor([[5.1, 3.5, 1.4, 0.2]])

# send input to the model and get the output
y_hat = model(X)
y_hat

You can see that every time we need to provide some numbers to PyTorch it is not enough just to combine them into a 1D list using the squared brackets, e.g.:

```python
X = [5.1, 3.5, 1.4, 0.2] 
```

or make 2D list like:

```python
X = [[5.1, 3.5, 1.4, 0.2]]
```

We also need to convert the list to a special type, `torch.tensor`. This is similar to what we used in the first class to create NumPy arrays, we provided values as a list and then used a special method that tells Python — make it NumPy array:

```python
X = np.array([[5.1, 3.5, 1.4, 0.2]])
```

From visual point of view *Tensor* is the same as array, it can be 1D (vector), 2D (matrix), 3D and so on. So, Torch tensor is similar to NumPy array you already know. But in reality there is a difference. For Python they are two different data objects from two different libraries, and you can apply different methods and operators to each. Therefore it is important to "tell" Python that this is not just a list or NumPy array, but a PyTorch tensor.

>One of the examples here can be cars and service centers. If you have Ford, you will not go to Hyundai's service center to e.g. change breaks and engine oil. Because their mechanics are not certified to work with this brand. Although both are cars, they look similar, work similar, and you can easily drive both, they are not identical.  

One of the reasons to use Torch tensors instead of NumPy arrays is that the tensors are designed to work with [GPU](https://en.wikipedia.org/wiki/Graphics_processing_unit) — the powerful graphic cards we have e.g. in gaming computers. Moreover it works only with specific GPU, made by company [NVIDIA](https://www.nvidia.com/da-dk/geforce/graphics-cards/). If you are gamer you probably heard about graphics card like GTX 3090. Because ANNs require a lot of computer power, using GPU makes them much faster and PyTorch is made to work on GPU first. If you do not have GPU, it can also work on conventional processor, but the computation will be slow. Especially if you have large datasets with thousands of objects.

So although it is a bit annoying that we have to add `torch.tensor()`, you will find out later that it is not a big problem.

If we get back to our code and its computed value, you can see, the output is not $6.01$, more over every time you run this code (try to click on *Run* button several times), you will get a new output. Because when you initialize the ANN model it uses random numbers for your weights. 

You can change this and assign weights manually after initialization:

In [ ]:
# define weights and bias
W = torch.tensor([[0.1, 0.2, 2.0, 5.0]])
Bias = torch.tensor([1.0])

# set weights and bias manually for the neurons on layer "layer1"
model.layer1.weight = nn.Parameter(W)
model.layer1.bias = nn.Parameter(Bias)

# forward pass through the model
y_hat = model(X)
y_hat

Now it is $6.01$! You can notice that the value we get as an output is also a tensor.

Ny the way with Torch you can compute linear combination also manually:

In [ ]:
(X * W).sum() + Bias


Let's now implement the classification rule for *Virginica* samples we created in a previous class. If you remember we compared Petal Width ($X_4$ in our case)  of a flower with 1.7 and if the value was above this threshold we classified this flower as *virginica*. 

These are the weights and bias which can implement this rule:

In [ ]:
# define weights and bias
W = torch.tensor([[0.0, 0.0, 0.0, 1.0]])
Bias = torch.tensor([-1.7])

# set weights and bias manually
model.layer1.weight = nn.Parameter(W)
model.layer1.bias = nn.Parameter(Bias)

# forward pass through the model
y_hat = model(X)

# show result and apply threshold
(y_hat, y_hat > 0)

As you can see, all weights except the last one are set to zero, while the weight for $X_4$ is equal to one. This gives the output, $\hat{Y}$ simply equal to the $X_4$ value, which in our case is Petal Width. After that, we apply bias, so we subtract our threshold. This gives the following:

$\hat{Y} = X_4 - 1.7$

Apparently if *Petal Width* is above the threshold, the output value will be negative, and when it is above, the value will be positive. So we can make classification decision by simply comparing it with 0, like we did above.

Now let's apply this network with manually set weights to the test set:

In [ ]:
# take only columns with measurements and convert them to Torch tensor
X_test_values = d_test.iloc[:, 1:5].values
X_test = torch.tensor(X_test_values).float()

# apply the model
y_hat = model(X_test)

# show the results
y_hat > 0

Indeed we got a lot of negative numbers at the beginning, where we have flowers of *Setosa* and *Versicolor* species, and positive numbers at the end, where we have the flowers of the target class, *virginica*.

Let's combine the output and the reference values to data frame for better visibility:

In [ ]:
# convert output to numpy array (transpose -> detach from model -> convert to NumPy array)
y_hat_arr = y_hat.t().detach().numpy()
y_hat_arr

In [ ]:

# combine with reference values
res = pd.DataFrame({
    "Reference": d_test["Species"],
    "Predicted": y_hat_arr[0],
    "Is virginica": y_hat_arr[0] > 0
})
res

We got absolutely the same results as in the last class! 

But how to let the model find the classification decision automatically, based on the provided data? We need to train the model! But what we should use as a training criterion?

## Loss function

The whole idea of ANN training is to find the weights (and biases and other parameters if any) for all neurons which will make the output as close to the desired as possible. 

But how to measure the distance between the output of the model and the desired output? This is what is defined as a *loss*. Loss is a number, a statistic, which tells how big the difference is between the desired output, $Y$, and the predicted one, $\hat{Y}$.

As you remember, we want to make classification for *Virginica* flowers. Let's define the ideal output to be 1 for *virginica* and -1 for the others. Here how to create it:


In [ ]:
# get the classes labels for the training set
c = d_test["Species"]

# compare the labels with target class then multiply the result to 2 and subtract 1
# if the result is False, it will be treated as 0: 0 * 2 - 1 = -1
# if the result is True, it will be treated as 1: 1 * 2 - 1 = 1
y_dummy = (c == "virginica") * 2 - 1
y_dummy

In [ ]:
# convert the values from the data frame column to torch array
y = torch.tensor([y_dummy.values])
y

The only problem we have is that the values are presented as rows and we need them as a column. So let's add transposition and also convert them from integer number to floating point numbers:

In [ ]:
# convert the values to torch array with 1 column and 120 rows
y = torch.tensor([y_dummy.values]).float().t()
y

Every time we get output from the model we need to compare it with these reference y-values and compute a statistic which tells how big the difference is — the *loss*. The simplest statistic which will do it for you is called mean squared error (MSE). In this case you simply take a difference between the two vector of values, square this difference and compute average (mean). 

Here is how to implement it:

In [ ]:
def mse_loss(y_hat, y):
    """ computes mean squared error loss """
    return ((y - y_hat)**2).mean()

Now let's test it. We already have the outputs and the desired (reference) values for the test set, so we can compute the loss value:

In [ ]:
loss_test = mse_loss(y_hat, y)
loss_test

The smaller loss the better. 

In fact you do not need to calculate the loss manually, like we did in the code block above. PyTorch has a lot of loss functions already implemented and specifically created to be used in the training process (e.g. for computing gradients). Here is the one for MSE:

In [ ]:
loss_function = nn.MSELoss()
loss_test = loss_function(y_hat, y)

loss_test

As you can see, it provides value identical to what we got using out own manual implementation of the MSE loss.

As we mentioned above, there are many different ways to measure the loss, so MSE is not the only one which is in use. For example, in case of binary classification another function, *Binary Correlation Entropy* (BCE) is used. In case of multiple class classification, one can use *Cross Entropy Loss*.

You do not need to know all of them, just remember that the loss function shows you (and to your model) how big the difference is between the desired output and the ones you model computes now. 

Moreover it tells model how to compute gradients — set of steps which let ANN to change the weights in order to make the loss smaller. This process of reducing the loss by gradual updating weights is called *[gradient descent](https://www.ibm.com/topics/gradient-descent)* and this is the main way to train any ANN model. Now we can discuss the training in detail.

### Exercise 1

Fill in the following two tables (you can do it in e.g. Excel or manually on a paper), compute MSE for each and comment how well MSE describes the classification results:


*Case 1*

| $y$ | $\hat{y}$ | $(y - \hat{y})$ | $(y - \hat{y})^2$ |
| --:| ---------:| ---------------:| -----------------:|
| -1 | 0.2 | - | - |
| -1 | 0.4 | - | - |
|  1 | -0.2 | - | - |
|  1 | -0.4 | - | - |

*Case 2*

| $y$ | $\hat{y}$ | $(y - \hat{y})$ | $(y - \hat{y})^2$ |
| --:| ---------:| ---------------:| -----------------:|
|  1 | 0.2 | - | - |
|  1 | 0.4 | - | - |
| -1 | -0.2 | - | - |
| -1 | -0.4 | - | - |


## Gradients and optimization 

Loss function is used not only for assessing the quality of the predicted values, but also for computing gradients for the weights — how the weights must be changed in order to get a smaller loss at the next iteration (improve the model). 

The gradients are increments for the weights, $\Delta w_1, \Delta w_2, \Delta w_3, \Delta w_4$ and for the bias, $\Delta b$, which are used to compute new weights for the model. The simplest way to compute the new weights is the following:


$w_1 = w_1 - \alpha \Delta w_1$<br>
$w_2 = w_2 - \alpha \Delta w_2$<br>
$w_3 = w_3 - \alpha \Delta w_3$<br>
$w_4 = w_4 - \alpha \Delta w_4$<br>
$bias = bias - \alpha \Delta b$

As you can see above, the increments are not used as is, but there is an additional parameter $\alpha$, which must be between 0 and 1. For example, if $\alpha = 0.01$, the change in weights  will be 1% of the computed increment.

The parameter $\alpha$ is called a *learning rate* and it is needed to slow down the learning process and make it more smooth. You will see some examples later in this class.

The gradients are computed by the loss function object (in example above it is `nn.MSELoss()`), while updating the weights is done by another function — *optimizer*. There are several optimizers available, usually one of the following two is a good choice:

* *GD* or *SGD* (stochastic gradient descent) — the simplest optimizer, which works exactly like shown in the equations above. Simple and straightforward. The name *gradient descent* means that you compute gradient in order to find steps for the weights which let go down (descent) in the loss.
* *Adam* is more sophisticated optimizer which updates weights in a bit more complex way than as shown above. It has more parameters to tune and usually is more efficient.

In all examples we will use *SGD* as it is more simple, but you can try *Adam* later. 



## Training ANN model

The idea of ANN training is as follows:

1. Apply ANN to compute outputs based on inputs from the training set.
2. Compute loss based on the current output and the desired output values.
3. Compute gradients and update weights, so next time the loss value is smaller.

The first step is called *forward propagation* because data values flow from the left (inputs) to the right (outputs) through all neurons in between (we have only one neuron so far in our model, but it does not matter — what works for one will work for thousands). 

The third step is called *back propagation*, because the gradients are computed for the output neuron first. Then for the neurons left to the output, and so on, until all neurons get the new weights updated based on the gradients.

These three steps are repeated until a criterion is met, for example loss does not get smaller anymore. Every time a model takes all three steps for all rows of the training set it takes an *epoch*. So if you run training process for 10 epochs, it means this three steps are repeated 10 times for all rows of the training set.

Let's implement this for our model. First of all let's prepare X and y values from out training set (so they are numeric and converted to Torch tensor)

In [ ]:
# select X variables, convert them to torch tensor and make them to have type float
X_train_values = d_train.iloc[:, 1:5].values
X_train = torch.tensor(X_train_values).float()

# compare species values with target class to get logical target values
c_train = d_train["Species"]

# convert logical values to numeric, so it is +1 if flower is virginica and -1 otherwise
y_train_dummy = (c_train  == "virginica") * 2.0 - 1.0

# convert the numeric values to torch tensor, make them float and transpose to a column
y_train = torch.tensor([y_train_dummy.values]).float().t()

Now let's define the number of epochs for training and the loss function:

In [ ]:
# number of epochs to train the model
nepochs = 20


# define a loss function
loss_function = nn.MSELoss()

How many epochs to use depends on the case, usually 100 epochs can be used as a starting point. We will use 20 just to make the output shorter.

We are ready to train the model.

Before we start we will fix the state of random number generator (which is used to initialize weights). This is needed to get reproducible results below, so when we re-run the code the results will be the same. If you want later to try this code with truly random weights simply comment these two lines.

In [ ]:

# fix the state of random numbers generator
seed = 11
#torch.manual_seed(seed)

# initialize a new model
model = SimpleModel()

# define optimizer which will compute gradients — do the learning, back propagation.
#
# parameter "lr" is learning rate it tells how large changes
# the weights will have (so it regulates how fast the learning process is)
# - if "lr" is too small your model can stuck and never reach the optimal model
# - if "lr" is too large your model can overshoot the optimal model
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# set model to training mode
model.train()

# training loop
for epoch in range(nepochs):

    # the gradients to zero
    optimizer.zero_grad()

    # 1. forward pass
    y_hat = model(X_train)

    # 2. compute the loss
    loss = loss_function(y_hat, y_train)

    # 3. backward pass and optimize weights
    loss.backward()
    optimizer.step()

    # show how big the loss is
    print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

If you comment the line which freezes random number generator and click on run button several times. You will see that the loss values are different every time you re-initialize and train your model. It is because the initial weights are set to random numbers, so training outcome is not pre-determine.

This is why for the learning purposes we need to freeze the state. Please remove the comment and make sure the results are reproducible.

Let's see how it performs on the training set


In [ ]:
# set model to predictions mode
model.eval()

# apply model to training set
y_hat = model.forward(X_train)

# convert output to numpy array
y_hat_arr = y_hat.t().detach().numpy()

# combine with reference values
res = pd.DataFrame({
    "Reference": d_train["Species"],
    "Predicted": y_hat_arr[0]
})

# compute statistics
TP = sum((res["Reference"] == "virginica") & (res["Predicted"] > 0))
TN = sum((res["Reference"] != "virginica") & (res["Predicted"] < 0))
FP = sum((res["Reference"] != "virginica") & (res["Predicted"] > 0))
FN = sum((res["Reference"] == "virginica") & (res["Predicted"] < 0))

sens = TP / (TP + FN)
spec = TN / (TN + FP)
acc = (TP + TN) / (TP + TN + FP + FN)

(sens, spec, acc)


Not bad for self training on 30 epochs. And since this is a very simple one neuron model you can get and look at the weights and bias for this neuron:

In [ ]:
# show weights and bias of the trained model
(model.layer1.weight, model.layer1.bias)

As you can see, the most important input is in this case is *Petal Length*, it has the biggest weight of $0.56$. The least important is the second one (*Sepal Width*) it has a weight of $-0.03$. The first input (*Sepal Length*) contribute negatively in this combination as well as the last one. 

In other words, our ANN implements the following linear model:

$\hat{Y} = -0.36 \times X_1 - 0.03 \times X_2 + 0.56 \times X_3 - 0.27 \times X_4 + 0.07$


Now we can apply our model to the test set.

In [ ]:
# prepare X and y values for the training set
X_test = torch.tensor(d_test.iloc[:, 1:5].values).float()

c_test = d_test["Species"]
y_test = (c_test == "virginica") * 2.0 - 1.0
y_test = torch.tensor([y_test.values]).float().t()

# apply model to test set
y_hat = model.forward(X_test)

# convert output to numpy array
y_hat_arr = y_hat.t().detach().numpy()

# combine with reference values
res = pd.DataFrame({
    "Reference": c_test,
    "Predicted": y_hat_arr[0]
})

# compute statistics
TP = sum((res["Reference"] == "virginica") & (res["Predicted"] > 0))
TN = sum((res["Reference"] != "virginica") & (res["Predicted"] < 0))
FP = sum((res["Reference"] != "virginica") & (res["Predicted"] > 0))
FN = sum((res["Reference"] == "virginica") & (res["Predicted"] < 0))

sens = TP / (TP + FN)
spec = TN / (TN + FP)
acc = (TP + TN) / (TP + TN + FP + FN)

(sens, spec, acc)

For the test set it works even better (perhaps because the test set is smaller).

### Exercise 2

Now play a bit with this code. Try to remove `torch.manual_seed()` instruction and run training several times. Try to increase the number of epochs and see how it influences the quality of classification both for training set and test set. Try to change the learning rate (make it smaller or larger). Get the best possible model and report the results.

## Activation function and neuron layers 

How to make the model even more efficient? Well the most obvious way is to increase the number of layers and connect them all together. However, the total number of weights in this case will multiply and you can end up with the model with thousands of weights to tune. In this case you need more objects in order to train, validate and test it properly.

The second way to make ANN model more efficient is to make it non-linear. The simplest way to introduce non-linearity is to supplement every linear neuron with what is called an *activation function*. 

The activation function changes the output depending on its value. The simples activation function is called **Rectified Linear Unit (ReLU)**. It works as follows, if the output is negative, it makes it equal to zero. But when output is positive it simply keeps it as is.

On the image below you can see two different activation functions, the blue one is ReLU:

<img src="./illustrations/ReLU_and_GELU.svg" style="width:300px">


Let's implement ANN with 3 layers. First layer will consists of 8 neurons. Every neuron will have 4 inputs and 1 output, so this layer will have 8 outputs. Second layer will consists of 4 neurons, each neuron has 8 inputs and 1 output, so this layer will have four outputs. Finally, the last layer will be the same as we have in our model now — one neuron with 4 inputs and 1 output. 

>**Note to teacher**<br>draw the architecture on a black board.

Neurons in the first two layers will also have ReLU activation function for the output.

Here is the implementation:

In [ ]:
import torch.nn.functional as F
import torch.nn as nn

class NewModel(nn.Module):
    """ class for three layers ANN """

    def __init__(self):
        super(NewModel, self).__init__()
        self.layer1 = nn.Linear(4, 8)  # first layer: 4 inputs and 8 outputs
        self.layer2 = nn.Linear(8, 4)  # second layer: 8 inputs and 4 outputs
        self.layer3 = nn.Linear(4, 1)  # first layer: 4 inputs and 1 output

    def forward(self, x):
        x = F.relu(self.layer1(x)) # pass inputs through the first layer and apply ReLU activation
        x = F.relu(self.layer2(x)) # pass output from layer 1 through the second layer + ReLU activation
        y_hat = self.layer3(x) # pass output from layer 2 through the third layer (no relu)
        return y_hat

And here is the code which implements the rest (training and testing). 

As you can see in this case for each epoch we compute loss for training set and for test set. So we can detect situation when model gets worse for the test set and stop. This process is called *validation* and it let's us avoid overtraining the model, when it works perfect for training set and bad for the test set. This situation is also called *overfitting*.

In [ ]:
# we use another manual seed here to get reproducible outcome
torch.manual_seed(42)

# number of epochs to train the model
nepochs = 300

# define a loss function
loss_function = nn.MSELoss()

# initialize the new model
model = NewModel()

# define optimizer which will compute gradients — do the learning.
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# training loop
for epoch in range(nepochs):  # Number of training epochs

    # train
    model.train()
    optimizer.zero_grad()
    y_hat_train = model(X_train)
    train_loss = loss_function(y_hat_train, y_train)
    train_loss.backward()
    optimizer.step()

    # validate
    model.eval()
    y_hat_test = model(X_test)
    test_loss = loss_function(y_hat_test, y_test)

    # show how big the loss is
    print(f'Epoch {epoch}, train loss: {train_loss.item():.4f} - test loss {test_loss.item():.4f}')

As you can see, we use more epochs in this case as the model is a bit more complex. 

Let's see how it performs on the test set:

In [ ]:
# set model to prediction mode
model.eval()

# apply model to test set
y_hat = model.forward(X_test)

# convert output to numpy array
y_hat_arr = y_hat.t().detach().numpy()

# combine with reference values
res = pd.DataFrame({
    "Reference": c_test,
    "Predicted": y_hat_arr[0]
})

# compute statistics
TP = sum((res["Reference"] == "virginica") & (res["Predicted"] > 0))
TN = sum((res["Reference"] != "virginica") & (res["Predicted"] < 0))
FP = sum((res["Reference"] != "virginica") & (res["Predicted"] > 0))
FN = sum((res["Reference"] == "virginica") & (res["Predicted"] < 0))

sens = TP / (TP + FN)
spec = TN / (TN + FP)
acc = (TP + TN) / (TP + TN + FP + FN)

(sens, spec, acc)

As we mentioned above, strictly speaking for this last step we have to use another set of samples, independent from what was used for training and validation. We break this rule here for illustration purposes, only because our data set is small.

We can visualize the predicted values in order to get better understanding:

In [ ]:

import matplotlib.pyplot as plt
plt.scatter(res["Reference"], res["Predicted"])
plt.plot(plt.xlim(), [0, 0], color = "black", linestyle="--")
plt.ylabel("Predicted output")

It must be noted that neither the network we use in the last example nor the selected loss function are specifically good for classification. But even with this selection the result looks good. Let's learn how to improve this.

## Multiclass classification 

One of the most common ways to make classification model with ANN is to use the output layer with several neurons — one for each class. So, for binary classification we will get two outputs, for classification among three classes — three and so on. But how to make the classification decision in this case and which loss function to use?

The idea is similar to voting. The decision is made by selection output with the largest value. For example, if predicted values in the output layer are [0.23, 0.89, -0.01], then the second output "wins", so the predicted label index will be 1 (remember in Python indices start from 0, so we have 0, 1 and 2 instead of 1, 2, and 3). Which means the following:

1. We need to have as many outputs in the last (output) layer as many classes we have.
2. We need to create a vector with reference class indices.
3. We need to use a special loss function which works best in this case.

Let's implement this by creating a model which will predict the class label for the Iris data. This time any of the three labels. Let's assume that label `"setosa"` will have index 0, `"versicolor"` will have index 1 and `"virginica"` will have index 2. 

Let's create a dictionary with the indices and labels:

In [ ]:
# create vector with classes, so we can get a label by its index
classes = ["setosa", "versicolor", "virginica"]

# create dictionary so we can get index by label
class_to_idx = {"setosa": 0, "versicolor": 1, "virginica": 2}

Now let's prepare the datasets. We already did this above, but let's do repeat this again and this time let's also create vector with reference class indices.

In [ ]:
# create tensor with predictor values
X_values = d.iloc[:, 1:5].values
X = torch.tensor(X_values).float()

# create tensor with class indices
label_values = [class_to_idx[label] for label in d["Species"]]
labels = torch.tensor(label_values).long()

# show it on the screen to check
labels

As you remember adding `float()` to tensor makes the values as floating point numbers. Adding `long()` makes them integer values. Having label indices as integer numbers is required by the loss function we are going to use.

Now let's split both tensors to training and test set like we did before:

In [ ]:
# generate indices of rows for training and test set
train_ind = d["Id"] % 5 != 0
test_ind = d["Id"] % 5 == 0

# select rows and label indices for the training set
X_train = X[train_ind, :]
labels_train = labels[train_ind]

# select rows and label indices for the test set
X_test = X[test_ind, :]
labels_test = labels[test_ind]

# show test set labels
labels_test

As you can see, this time instead of making subsets for the data frame we created tensors first and then subset the tensors. This way is a bit shorter and perhaps more clear.

Now let's create a new class with ANN model with three outputs:

In [ ]:
import torch.nn.functional as F
import torch.nn as nn

class MultiClassModel(nn.Module):
    """ class for three layers ANN """

    def __init__(self):
        super(MultiClassModel, self).__init__()
        self.layer1 = nn.Linear(4, 8)  # first layer: 4 inputs and 8 outputs
        self.layer2 = nn.Linear(8, 16)  # first layer: 8 inputs and 16 outputs
        self.layer3 = nn.Linear(16, 3)  # first layer: 16 inputs and 3 outputs

    def forward(self, x):
        x = F.relu(self.layer1(x)) # pass inputs through the first layer and apply ReLU activation
        x = F.relu(self.layer2(x)) # pass output from layer 1 through the second layer + ReLU activation
        y_hat = self.layer3(x) # pass output from layer 2 through the third layer (no relu)
        return y_hat

As you can see the model is very similar to what we had before, we just changed the number of inputs/outputs. 

>**Note to teacher:**<br>draw the model schematically on a black board.

Now we need to define the loss function. In this particular case, when the decision is made by voting the best suitable function is cross-entropy loss:

In [ ]:
# define a loss function
loss_function = nn.CrossEntropyLoss()

Everything is ready, let's initialize and train the model (as in the examples above we will freeze the random number generator state to get reproducible results):

In [ ]:
# seed the random number generator to get reproducible outcome
torch.manual_seed(12)

# initialize the new model
model = MultiClassModel()

# define optimizer which will compute gradients — do the learning.
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# number of epochs to train the model
nepochs = 100

for epoch in range(nepochs):  # Number of training epochs

    # train
    model.train()
    optimizer.zero_grad()
    labels_predicted = model(X_train)

    loss = loss_function(labels_predicted, labels_train)

    loss.backward()
    optimizer.step()
    train_loss = loss.item()

    # validate
    model.eval()
    labels_predicted = model(X_test)
    loss = loss_function(labels_predicted, labels_test)
    val_loss = loss.item()

    # show how big the loss is
    print(f'Epoch {epoch}, train loss: {train_loss:.4f} - validation loss {val_loss:.4f}')


So far so good, although as you can see the validation loss continues to decrease, so perhaps 100 epochs is not enough. We will come back to this later. Let's learn how to make predictions.

Let's make predictions for both sets and look at the predicted values for the test set first:

In [ ]:
# set model to prediction mode
model.eval()

# apply model to train and test sets
output_train = model.forward(X_train)
output_test = model.forward(X_test)

# show predictions for the test set
output_test

If you look carefully you can find out that indeed for the first ten rows (where we have *setosa* samples), first out of the three values is the largest. For the other two it is not so clear. Let's apply `max` function to each row and ask not the value but the position (index) where the largest value is located:

In [ ]:
# get indices of largest values for each row of computed outputs
_, predictions_train = torch.max(output_train, 1)
_, predictions_test = torch.max(output_test, 1)

predictions_test

Indeed the predicted class label index for setosa is good, but the others are not perfect so far. We will fix it later. 

Because here we have three classes, it will be to laborious to compute classification statistics for each. Let's learn a new way to assess classification results — via contingency table or [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix). It simply shows all possible combinations of the reference and the predicted class labels:

In [ ]:
import numpy as np

# compute contingency table for training and test set
ct_train = np.zeros((3, 3))
ct_test = np.zeros((3, 3))

for i in range(3):
    for j in range(3):
        ct_train[i, j] = sum((labels_train == i) & (predictions_train == j))
        ct_test[i, j] = sum((labels_test == i) & (predictions_test == j))

(ct_train, ct_test)


The table for ideal classification should have zero for all values except the diagonal (where predicted and reference label index match). As you can see we have perfect match for the first (0, "setosa") and the second (2, "versicolor") classes but the "virginica" class is not predicted well. Six flowers of this class were wrongly predicted as *versicolor* and four were correctly predicted as *virginica*.

You can also compute this table for relative values (per cent), which is easier to use when the number of individuals in different classes is not the same:

In [ ]:
# compute contingency table for training and test set with relative values
ct_train = np.zeros((3, 3))
ct_test = np.zeros((3, 3))
for i in range(3):
    n_train = sum(labels_train == i)
    n_test = sum(labels_test == i)
    for j in range(3):
        ct_train[i, j] = sum((labels_train == i) & (predictions_train == j)) / n_train
        ct_test[i, j] = sum((labels_test == i) & (predictions_test == j)) / n_test

(ct_train, ct_test)


Let's learn how to visualize this by making a heatmap.

In [ ]:
# heatmap plot for contrast table
plt.imshow(ct_test, clim = [0, 1])
plt.colorbar()

Now let's add class labels and show the numbers for each cell of the table:

In [ ]:
# heatmap plot for contrast table
plt.imshow(ct_test, clim = [0, 1])
plt.colorbar()
plt.gca().set_xticks(range(3), classes)
plt.gca().set_yticks(range(3), classes)
for i in range(3):
    for j in range(3):
        plt.text(i, j, round(ct_test[j, i], 3), color = "white" if ct_test[j, i] < 0.5 else "black")

Let's create several functions in order to re-use the code we have written above.

In [ ]:
def train(model, X_train, labels_train, X_val, labels_val, nepochs = 100, lr = 0.01):
    """ trains any classification model using provided data, number of epochs and learning rate """

    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    for epoch in range(nepochs):
        model.train()
        optimizer.zero_grad()
        labels_predicted = model(X_train)
        loss = loss_function(labels_predicted, labels_train)
        loss.backward()
        optimizer.step()
        train_loss = loss.item()

        model.eval()
        labels_predicted = model(X_val)
        loss = loss_function(labels_predicted, labels_val)
        val_loss = loss.item()

        print(f'Epoch {epoch}, train loss: {train_loss:.4f} - validation loss {val_loss:.4f}')

In [ ]:
def predict(model, X):
    """ get ANN model and tensor with predictors and returns predicted class label indices """
    model.eval()
    output = model.forward(X)
    _, labels = torch.max(output, 1)
    return labels

In [ ]:
def table(reference, predicted):
    """ computes contingency table for predicted and reference class label indices """
    indices = reference.unique()
    n = len(indices)
    ct = np.zeros((n, n))
    for i in range(n):
        ni = sum(reference == indices[i])
        for j in range(n):
            ct[i, j] = sum((reference == indices[i]) & (predicted == indices[j])) / ni

    return ct

In [ ]:
def ct_heatmap(ct, classes):
    """ shows heatmap for the contingency table """
    plt.imshow(ct, clim = [0, 1])
    plt.colorbar()

    n = len(classes)
    plt.gca().set_xticks(range(n), classes)
    plt.gca().set_yticks(range(n), classes)
    for i in range(n):
        for j in range(n):
            plt.text(i, j, round(ct[j, i], 3), color = "white" if ct[j, i] < 0.5 else "black")

As you can see, the function `train()` does not return anything. It is because `model`, which we pass to this function as a first argument is object, it has its own methods which update this object internally. So although all training happens inside this function, the object outside gets all necessary update.

Moreover, we can use this function for any other models as well.

Let's check how the new functions work. Let's train re-initialite the model and train it again using more epochs and smaller learning rate:

In [ ]:
torch.manual_seed(12)
model = MultiClassModel()
train(model, X_train, labels_train, X_test, labels_test, nepochs = 1000, lr = 0.01)

And check how it performs:

In [ ]:
predictions_train = predict(model, X_train)
predictions_test = predict(model, X_test)

ct_train = table(labels_train, predictions_train)
ct_test = table(labels_test, predictions_test)

plt.figure(figsize = (10, 5))

plt.subplot(1, 2, 1)
ct_heatmap(ct_train, classes)
plt.title("Train")

plt.subplot(1, 2, 2)
ct_heatmap(ct_test, classes)
plt.title("Test")

With 1000 epochs, smaller learning rate the new architecture works almost perfectly!

### Exercise 3

In file `IrisHeatmap.csv` you will find new data points which contain new measurements but do not have column with species nor column with IDs (so it has only four columns with measurements). Load the data from the file and then apply the ANN you have just trained to get the predictions.

After that make a scatter plot, where x- and y-values are Petal Length and Petal Width you got from the file. Show points, predicted as *setosa* using red color, points, predicted as *versicolor* as green and points, predicted as *virginica* as blue. Comment the plot. What it actually shows? 

Try to add predictions for training and test set to the plot. In this case you will need to make the predictions for new data shown semi-transparent. Use parameter `alpha = 0.15` in the `plt.scatter()` function for that.


## Visualization of the training process

It can be useful to see how training and validation loss changes with epochs. Let's modify the `train_model()` method in order to collect this information and return it back to user:

In [ ]:
def train(model, X_train, labels_train, X_test, labels_test, nepochs = 100, lr = 0.01):
    """ trains any classification model using provided data, number of epochs and learning rate """

    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    train_losses = np.zeros(nepochs)
    val_losses = np.zeros(nepochs)

    for epoch in range(nepochs):
        model.train()
        optimizer.zero_grad()
        labels_predicted = model(X_train)
        loss = loss_function(labels_predicted, labels_train)
        loss.backward()
        optimizer.step()

        train_losses[epoch] = loss.item()

        model.eval()
        labels_predicted = model(X_test)
        loss = loss_function(labels_predicted, labels_test)
        val_losses[epoch] = loss.item()

        print(f'Epoch {epoch}, train loss: {train_losses[epoch]:.4f} - validation loss {val_losses[epoch]:.4f}')

    return (train_losses, val_losses)

Now let's train the model again, this time using 5000 epochs, get the loss values and make a plot.

In [ ]:
# seed the random number generator to get reproducible outcome
torch.manual_seed(12)

# re-initialize and train the model
model = MultiClassModel()
train_losses, val_losses = train(model, X_train, labels_train, X_test, labels_test,
                                 nepochs = 5000, lr = 0.01)

In [ ]:
# show plot with both losses
plt.plot(train_losses, label = "train")
plt.plot(val_losses, label = "val")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.ylim([0, 0.2])

As you can see, after approximately 3000 epochs the validation almost loss does not change. So there is no reason to run the model that far.

Let's re-initialize the model (this is needed to set all initial weights to random numbers otherwise weights for already trained model will be used as starting point) and train it again, but this time we will use a large learning rate:

In [ ]:
torch.manual_seed(12)
model = MultiClassModel()
train_losses, val_losses = train(model, X_train, labels_train, X_test, labels_test,
                                 nepochs = 5000, lr = 0.2)

In [ ]:
plt.plot(train_losses, label = "train")
plt.plot(val_losses, label = "val")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")

You can see some strange patterns, "peaks", on the plot. These patterns are typical for the large learning rate, when the optimizer changes weights of the neurons too much, so the model "jumps" back and force. 

This can be compared with driving a car. If you accelerate and brake slowly, monitoring the situation on the road, and take actions in advance (pro-active), your car will move smoothly and comfortable for you and your passenger. But the situation we see on the plot above corresponding to the case when you accelerate and break every minute.

This plot helps to find the optimal learning rate and number of epochs. 

## Save and load the model state

Finally let's learn how to save and load the model state — all internal parameters, which include weights and biases of all layers, etc. This can be handy in the following situations:

1. You can save the model state into a separate variable after every epoch, but only if validation loss gets better. This helps to avoid overfitting, when you train your model too long and it performs worse on the validation set than e.g. 50 epochs before. We will consider this case in the next class.

2. Save the model to a file for later use. For example you can send it to your classmates or colleagues. Or simply save it in order to continue training process tomorrow. Or to re-use it for predictions. Many possibilities.

Let's see first how to get the state of the model. Try to run the next code first:

In [ ]:
model.parameters()

In [ ]:
for parameter in model.parameters():
    print(parameter.shape)

You can see a collection of torch tensors (similar to NumPy arrays as we remember). For example, first tensor has 8 rows and 4 columns. Can you guess why?

Because in the first layer of your model you have 8 neurons. Every neuron has 4 inputs and 1 output. And output is computed by taking a weighted sum of the inputs plus bias. So for every neuron you need 5 parameters — 4 weights and 1 bias. 

The first tensor of size 8x4 contains the weight for every of the 8 neurons and the second tensor (vector with 8 values) contains the biases.

The second layer has 16 neurons, every neuron has 8 inputs and 1 output. So every neuron has 9 parameters — 8 weights and 1 bias. This is exactly what you have in the next pair of tensors, the one of size 16x8 contains weights and the one with 16 values contains the biases.

Finally the last layer has 3 neurons with 16 inputs and 1 output each, so the last two tensors contain weights and biases for these neurons.

In fact you can see the structure of your model and number of parameters by using method `summary()` from library `torchinfo`:


In [ ]:
from torchinfo import summary
summary(model)

You can also compute the total number of parameters of your model manually by simply taking a sum of all elements in each tensor:

In [ ]:
npar = 0
for parameter in model.parameters():
    npar = npar + parameter.numel()

print(f"Total number of parameters in this ANN: {npar}")

You can of course see all the weights (the output will be long though):

In [ ]:
for parameter in model.parameters():
    print("------")
    print(parameter)

Strictly speaking you can take all these values, copy-paste them to Excel and run predictions in Excel (you just need to remember to apply ReLU function to each output). Crazy idea but it is doable. This shows that ANN is not a rocket science, but very simple and straightforward yet powerful method.

There is another way to get the model parameters with all necessary additional information — state dictionary. There is a method for that:

In [ ]:
model.state_dict()

As you can see the output is similar to what we have seen before, but this time it is organized as a dictionary, so Torch will know a level name and the parameter name.

We can save the state to a variable by taken a deep copy:

In [ ]:
# we need to load a special function which creates copy of complex objects
from copy import deepcopy

# save parameters of current model to a variable
model_state = deepcopy(model.state_dict())

Why you need to take a deep copy instead of just assigning the state to a new variable? Because if you continue training your model this state will also get all updates. By taking a deep copy you kind of disjoint the current state from the next ones, make it independent.

Here is an example how you can use it:

In [ ]:
# seed the random number generator to get reproducible outcome
torch.manual_seed(12)

# initialize a new model
new_model = MultiClassModel()

# make predictions — they will be very bad because the model is not trained
predictions = predict(new_model, X_test)
predictions

In [ ]:
# now lets load the parameters we saved from the trained model to this new model
new_model.load_state_dict(model_state)

# and make predictions
predictions = predict(new_model, X_test)
predictions

And now we got perfect predictions without training the new model but just by reusing the parameters of the previously trained model.

If you want to save the state to file and send it to someone or load it later in another Python script, you can use two PyTorch functions: `torch.save()` saves model dictionary to a file and `torch.load()` loads it from the file. The file should have extension `.pth`. 

Let's see how it works:

In [ ]:
# save state dictionary to file
torch.save(model.state_dict(), "mymodel.pth")

As you can see we do not need to take a deepcopy in this case because it will be in separate file. If you run it you will get file `mymodel.pth` inside the current folder.

There is no reason to open it as inside the information is coded using binary format. But you can load it and assign to model:

In [ ]:
# create a new model with random weights
another_model = MultiClassModel()

# load state from the file and assign it to the new model
model_state = torch.load("mymodel.pth")
another_model.load_state_dict(model_state)

# make predictions
predictions = predict(another_model, X_test)
predictions

It works!

### Exercise 

To make this exercise you need to work in pairs. One of the member should train ANN network on Iris data (make a separate notebook and copy all needed code there). Train it using random initialization and try different hyperparameters (number of epochs, learning rate, etc.). When you get a good model save it to a file and send it to your group mate by email. The task of the group mate is to load the model and apply it to the test set.


In [ ]:
# place your code here



## Play with interactive ANN in web

Now you know everything you need to know for the next class. However we recommend you to play a bit more using this interactive ANN constructor which you can run directly in your web-browser. Spend some time and play with different problems, different architectures and hyperparameters:

[TensorFlow playground](http://playground.tensorflow.org)